In [1]:
import pandas as pd
%matplotlib widget

import copy
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy import cluster
from scipy.integrate import cumtrapz,simps
import scipy.interpolate
from sklearn.cluster import KMeans
from scipy.interpolate import UnivariateSpline

from Funciones_auxiliares import plot_branch,get_cluster_indexs,plots_hierarchichal_clustering
plt.close('all')

In [2]:
data = '//home/lorenzo/Principal/Noise-Neuron discriminator/Datos/PCCIBNMZc'
data = pd.read_pickle(data)
#df = data[data.PatientExperiment.str.contains('HEC032|HEC031|HEC030')]
#df.reset_index(inplace = True)
Neurons = data[data.bNoise == 0]
Neurons.reset_index(inplace = True)

In [3]:
'''Ejemplo de media neuronal'''
neuron = Neurons.loc[32]
plt.plot(neuron.Mean)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
df = Neurons.copy()
fig,axes,fl = plots_hierarchichal_clustering(df)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
Cluster_selected = get_cluster_indexs(5,fl)
Emb = Neurons.loc[Cluster_selected]
x = np.concatenate(Emb.Mean.values)
x = x[0:64]
x = (x - np.mean(x))/np.std(x)
x = x - x[0]


In [6]:

t = np.linspace(0,len(x) - 1.0001,10000)
t2 = np.arange(len(x))
x_interp = UnivariateSpline(t2,x,s = 0)
x_punto_interp = x_interp.derivative()
x_puntopunto_interp = x_interp.derivative(2)

x_interp = x_interp(t)
x_punto_interp = x_punto_interp(t)
x_puntopunto_interp = x_puntopunto_interp(t)


x_punto = np.gradient(x)
x_puntopunto = np.gradient(x_punto)


x_punto_integrated = cumtrapz(x_puntopunto)
x_integrated = cumtrapz(x_punto_integrated)

x_punto_integrated_interp= cumtrapz(x_puntopunto_interp)
x_integrated_interp= cumtrapz(x_punto_integrated_interp)

fig = plt.figure()
axes = fig.subplots(2,2)
axes[0][0].scatter(t2,x,label = 'senial original')
axes[0][0].plot(x_integrated,color = 'r',label = 'senial integrada')
axes[0][1].scatter(t,x_interp,color = 'g', marker = '.',label = 'senial interpolada')
axes[0][1].plot(t[1:-1],x_integrated_interp,color = 'c',label = 'senial interpolada integrada')

axes[1][0].plot(x_punto,label = 'x_punto')
axes[1][0].plot(x_puntopunto,label = 'x_puntopunto')
axes[1][1].plot(t,x_punto_interp,label = 'x_punto interp')
axes[1][1].plot(t,x_puntopunto_interp,label = 'x_puntopunto interp')
axes[0][0].legend()
axes[0][1].legend()
axes[1][0].legend()
axes[1][1].legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
fig = plt.figure()
#ax = fig.gca(projection = '3d')
#ax.plot(x_interp,x_punto_interp)
#ax.set_xlabel('x')
#ax.set_y
#label('$\dot{x}$')
plt.plot(x_interp,x_punto_interp)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
Neurons.head()

,index,PatientExperiment,Channel,Cluster,ISI,Bulk,bNoise,Mean,ZeroCrossings
0,3,HEC016\016context1\016context1,1,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[[-9.910404058695178, -10.706527972656263, -13...",0,"[5.549115009633331, 6.0470913803953605, 5.9916...",2
1,12,HEC016\016context1\016context1,2,2,"[0, 0, 1, 0, 0, 2, 0, 1, 1, 3, 2, 1, 3, 1, 2, ...","[[13.198738217761697, 11.760198708498171, 8.78...",0,"[13.243695374870788, 13.079745522544341, 12.71...",2
2,34,HEC017\017context2\017context2_prescr,113,2,"[0, 3, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, ...","[[18.892933610485304, 15.485716617075768, 8.32...",0,"[6.993260836407021, 6.865110211253, 6.51757150...",2
3,35,HEC017\017context2\017context2_prescr,113,3,"[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[[8.607666061103055, 4.40269092990618, 2.82740...",0,"[2.722673147546738, 2.3590640477204063, 2.0802...",6
4,47,HEC017\017context2\017context2_prescr,118,3,"[0, 4, 1, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[[5.702971353263122, 5.288244296823778, 3.4659...",0,"[4.340277642514026, 4.058565844662801, 3.92196...",2


# Sindy

Definimos Theta

In [9]:
x = x_interp
x_punto = x_punto_interp
x_puntopunto = x_puntopunto_interp
degree = 10
idx = np.arange(1,degree,2)
Theta = np.zeros([x.shape[0],degree])
Theta[:,0] = np.ones(x.shape[0])
Theta[:,1] = x
Theta[:,2] = x_punto
Theta[:,3] = x*x_punto
Theta[:,4] = x_punto**2
Theta[:,5] = x**3
Theta[:,6] = x**2*x_punto
Theta[:,7] = x**2
Theta[:,8] = x_punto**3
Theta[:,9] = x**4

Lasso

In [10]:
from sklearn.linear_model import Lasso
from scipy.integrate import odeint

classifier = Lasso(alpha=0.000000000001,max_iter=1000000)
classifier.fit(Theta,x_puntopunto)
print(Theta.shape,len(x_puntopunto))

(10000, 10) 10000


Vector de coeficientes ($\eta$)

In [105]:
coef_vector = copy.copy(classifier.coef_)
print(coef_vector)

[ 0.          0.04257972  0.29132122  0.20846416 -0.13324995 -0.05847057
 -0.01316826 -0.06451597 -0.11427324 -0.00517077]


Ecuaciones diferenciales

In [12]:
def f(X,t):
    r,rpunto = X
    u = np.array([1,r,rpunto,r*rpunto,rpunto**2,r**3,rpunto*r**2,r**2,rpunto**3,r**4])
    
    drdt = rpunto
    drpunto_dt = np.dot(coef_vector,u)
    return drdt,drpunto_dt

Integramos las ecuaciones y graficamos

In [13]:
X0 = (x[0],x_punto[0])
print('Initial conditions: X = ',X0[0],' X_punto = ',X0[1])
t = np.linspace(0,64,100000)
t2 = np.linspace(0,64,len(x))
tray = odeint(f,X0,t).T

fig = plt.figure()

plt.plot(t,tray[0])
plt.plot(t2,x)


Initial conditions: X =  1.3447872870528276e-20  X_punto =  0.0009942504213135525


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
t = np.linspace(0,64,100000)
t2 = np.linspace(64,0,1000)

X0 = (0,.010)
tray = odeint(f,X0,t).T

X1 = (-1,.01)
tray1 = odeint(f,X1,t).T

X2 = (.1,.01)
tray2 = odeint(f,X2,t).T

X3 = (.2,.01)
tray3 = odeint(f,X3,t).T

X4 = 0.,.01
tray4 = odeint(f,X4,t2).T

plt.figure()
plt.plot(tray[0],tray[1])
plt.plot(tray1[0],tray1[1])
plt.plot(tray2[0],tray2[1])
plt.plot(tray3[0],tray3[1])
plt.plot(tray4[0],tray4[1])

a = tray4[:,-1]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
coef_vector = copy.copy(classifier.coef_)
#coef_vector[2] = .28
#coef_vector[1] = 0.043
#coef_vector[3] = .235
#coef_vector[8] = -.1
t = np.linspace(0,100,10000)
cross = (-0.03,0.056)
ic = [cross + 0.00*(np.random.random(2) - 1/2) for i in range(1)]
plt.close('all')
plt.figure(90,figsize = (10,10))
for i in ic:
    tray = odeint(f,i,t).T
    plt.plot(tray[0],tray[1])
plt.plot(x,x_punto,'red')
plt.scatter(cross[0],cross[1],color = 'red')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [109]:
cross = (-0.03, 0.056)
t = np.linspace(0,64,10000)
coef_vector = copy.copy(classifier.coef_)
plt.figure(figsize = (10,10))
#Movemos mu1
mu1_list = np.linspace(0.055283522241592,0.055283522241593,10)
for mu1 in mu1_list:
    coef_vector[1] = mu1
    tray = odeint(f,cross,t).T
    plt.plot(t,tray[0],label = '$\mu$ = {}'.format(mu1))
plt.plot(t,x)
plt.legend()
cross

/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-0.03, 0.056)

In [107]:
cross = (-0.03, 0.056)
t = np.linspace(0,64,10000)
plt.figure(figsize = (15,15))
coef_vector = copy.copy(classifier.coef_)
#Movemos mu2 con mu1 = 0.55
coef_vector2[1] = 0.055283522241592
mu2_list = np.linspace(0.29099293808833,0.3,5)
for mu2 in mu2_list:
    coef_vector[2] = mu2
    tray = odeint(f,cross,t).T
    plt.plot(t,tray[0],label = '$\mu$ = {}'.format(mu2))
plt.plot(t,x)
plt.legend()
cross

/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-0.03, 0.056)

In [99]:
cross = (-0.03, 0.056)
t = np.linspace(0,64,10000)
plt.figure(figsize = (15,15))
#Movemos mu2 con mu1 = 0.55,mu2 = 0.291
coef_vector[1] = .055
coef_vector[2] = 0.29099293808847
mu3_list = np.linspace(0.207,0.209,10)
for mu3 in mu3_list:
    coef_vector[3] = mu3
    tray = odeint(f,cross,t).T
    plt.plot(t,tray[0],label = '$\mu$ = {}'.format(mu3))
plt.plot(t,x)
plt.legend()
cross

/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  This is separate from the ipykernel package so we can avoid doing imports until


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-0.03, 0.056)

In [23]:
cross = (-0.03, 0.056)
t = np.linspace(0,100,10000)
plt.figure(figsize = (15,15))
#Movemos mu2 con mu1 = 0.55,mu2 = 0.291,mu3 = 0.2083
coef_vector[1] = .055
coef_vector[2] = .291
coef_vector[3] = 0.2083
mu4_list = np.linspace(-0.141,-0.144,10)
for mu4 in mu4_list:
    coef_vector[4] = mu4
    tray = odeint(f,cross,t).T
    plt.plot(t,tray[0],label = '$\mu$ = {}'.format(mu4))
plt.plot(t,x)
plt.legend()
cross

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-0.03, 0.056)

In [20]:
cross = (-0.03, 0.056)
t = np.linspace(0,100,10000)
plt.figure(figsize = (15,15))
#Movemos mu2 con mu1 = 0.55,mu2 = 0.291,mu3 = 0.2083,mu4 = -0.16
coef_vector[1] = .055
coef_vector[2] = .291
coef_vector[3] = 0.2083
coef_vector[4] = -0.1413

mu5_list = np.linspace(-0.05848,-0.05847,10)
for mu5 in mu5_list:
    coef_vector[5] = mu5
    tray = odeint(f,cross,t).T
    plt.plot(t,tray[0],label = '$\mu$ = {}'.format(mu5))
plt.plot(t,x)
plt.legend()
cross

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-0.03, 0.056)

In [21]:
cross = (-0.03, 0.056)
t = np.linspace(0,64,10000)
plt.figure(figsize = (15,15))
#Movemos mu2 con mu1 = 0.55,mu2 = 0.291,mu3 = 0.2083,mu4 = -0.16,mu5 = -0.058477
coef_vector[1] = .055
coef_vector[2] = .291
coef_vector[3] = 0.2083
coef_vector[4] = -0.1413
coef_vector[5] = -0.058477
mu6_list = np.linspace(-0.01317,-0.013166,10)
for mu6 in mu6_list:
    coef_vector[6] = mu6
    tray = odeint(f,cross,t).T
    plt.plot(t,tray[0],label = '$\mu$ = {}'.format(mu6))
plt.plot(t,x)
plt.legend()
cross

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-0.03, 0.056)

In [22]:
cross = (-0.03, 0.056)
t = np.linspace(0,64,10000)
plt.figure(figsize = (15,15))
#Movemos mu2 con mu1 = 0.55,mu2 = 0.291,mu3 = 0.2083,mu4 = -0.16,mu5 = -0.058477,mu6 = -0.013169
coef_vector[1] = .055
coef_vector[2] = .291
coef_vector[3] = 0.2083
coef_vector[4] = -0.1413
coef_vector[5] = -0.058477
coef_vector[6] = -0.013169
mu7_list = np.linspace(-0.0645167,-0.06451,10)
for mu7 in mu7_list:
    coef_vector[7] = mu7
    tray = odeint(f,cross,t).T
    plt.plot(t,tray[0],label = '$\mu$ = {}'.format(mu7))
plt.plot(t,x)
plt.legend()
cross

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(-0.03, 0.056)

In [385]:
plt.figure()
plt.plot(x,x_punto)
plt.plot(tray[0],tray[1])

/home/lorenzo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  """Entry point for launching an IPython kernel.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
x = x_interp
x_punto = x_punto_interp
x_puntopunto = x_puntopunto_interp
degree = 7
idx = np.arange(1,degree,2)
Theta = np.zeros([x.shape[0],degree])
Theta[:,0] = np.ones(x.shape[0])
Theta[:,1] = x
Theta[:,2] = x_punto
Theta[:,3] = x*x_punto
Theta[:,4] = x_punto**2
Theta[:,5] = x**3
Theta[:,6] = x**2*x_punto

In [19]:
classifier = Lasso(alpha=0.000000001,max_iter=10000000)
classifier.fit(Theta,x_puntopunto)
print(Theta.shape,len(x_puntopunto))

(10000, 7) 10000


In [20]:
coef_vector = classifier.coef_
print(coef_vector)

[ 0.         -0.09843611 -0.11180471  0.29717996 -0.18435317 -0.01180283
  0.04806139]


In [21]:
def f(X,t):
    r,rpunto = X
    u = np.array([1,r,rpunto,r*rpunto,rpunto**2,r**3,rpunto*r**2])
    
    drdt = rpunto
    drpunto_dt = np.dot(coef_vector,u)
    return drdt,drpunto_dt

In [22]:
X0 = (x[0],x_punto[0])
print('Initial conditions: X = ',X0[0],' X_punto = ',X0[1])
t = np.linspace(0,64,100000)
t2 = np.linspace(0,64,len(x))
tray = odeint(f,X0,t).T

fig = plt.figure()

plt.plot(t,tray[0])
plt.plot(t2,x)


Initial conditions: X =  1.3447872870528276e-20  X_punto =  0.0009942504213135525


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
t = np.linspace(0,64,10000)
t2 = np.linspace(90,0,10000)
X0 = (-1.1,.010)
tray = odeint(f,X0,t).T

X1 = (-1,.01)
tray1 = odeint(f,X1,t).T

X2 = (-.9,.01)
tray2 = odeint(f,X2,t).T

X3 = (-2,-1)
tray3 = odeint(f,X3,t).T

x4 = tray[:,-1]
tray4 = odeint(f,x4,t2).T


plt.figure()
plt.plot(tray[0],tray[1])
plt.plot(tray1[0],tray1[1])
plt.plot(tray2[0],tray2[1])
plt.plot(tray3[0],tray3[1])
plt.plot(tray4[0],tray4[1])

plt.plot(x,x_punto)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Probemos con otra neurona

In [24]:
Cluster_selected = get_cluster_indexs(10,fl)
Emb = Neurons.loc[Cluster_selected]
x = np.concatenate(Emb.Mean.values)
x = x[1:64]
x = (x - np.mean(x))/np.std(x)
x = x - x[0]


In [25]:

t = np.linspace(0,len(x),100000)
t2 = np.arange(len(x))
x_interp = UnivariateSpline(t2,x,s = 0)
x_punto_interp = x_interp.derivative()
x_puntopunto_interp = x_interp.derivative(2)

x_interp = x_interp(t)
x_punto_interp = x_punto_interp(t)
x_puntopunto_interp = x_puntopunto_interp(t)


x_punto = np.gradient(x)
x_puntopunto = np.gradient(x_punto)


x_punto_integrated = cumtrapz(x_puntopunto)
x_integrated = cumtrapz(x_punto_integrated)

x_punto_integrated_interp= cumtrapz(x_puntopunto_interp)
x_integrated_interp= cumtrapz(x_punto_integrated_interp)

fig = plt.figure()
axes = fig.subplots(2,2)
axes[0][0].scatter(t2,x,label = 'senial original')
axes[0][0].plot(x_integrated,color = 'r',label = 'senial integrada')
axes[0][1].scatter(t,x_interp,color = 'g', marker = '.',label = 'senial interpolada')
axes[0][1].plot(t[1:-1],x_integrated_interp,color = 'c',label = 'senial interpolada integrada')

axes[1][0].plot(x_punto,label = 'x_punto')
axes[1][0].plot(x_puntopunto,label = 'x_puntopunto')
axes[1][1].plot(t,x_punto_interp,label = 'x_punto interp')
axes[1][1].plot(t,x_puntopunto_interp,label = 'x_puntopunto interp')
axes[0][0].legend()
axes[0][1].legend()
axes[1][0].legend()
axes[1][1].legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
x = x_interp
x_punto = x_punto_interp
x_puntopunto = x_puntopunto_interp
degree = 10
idx = np.arange(1,degree,2)
Theta = np.zeros([x.shape[0],degree])
Theta[:,0] = np.ones(x.shape[0])
Theta[:,1] = x
Theta[:,2] = x_punto
Theta[:,3] = x*x_punto
Theta[:,4] = x_punto**2
Theta[:,5] = x**3
Theta[:,6] = x**2*x_punto
Theta[:,7] = x**2
Theta[:,8] = x_punto**3
Theta[:,9] = x**4

Lasso

In [27]:
from sklearn.linear_model import Lasso
from scipy.integrate import odeint
classifier = Lasso(alpha=0.000001,max_iter=1000000)
classifier.fit(Theta,x_puntopunto)
print(Theta.shape,len(x_puntopunto))

(100000, 10) 100000


Vector de coeficientes ($\eta$)

In [28]:
coef_vector = classifier.coef_
print(coef_vector)

[ 0.          0.05155227  0.44021962  0.07512939 -0.43876236 -0.04946516
 -0.03408778 -0.0546355  -0.81142663 -0.00512404]


Ecuaciones diferenciales

In [29]:
def f(X,t):
    r,rpunto = X
    u = np.array([1,r,rpunto,r*rpunto,rpunto**2,r**3,rpunto*r**2,r**2,rpunto**3,r**4])
    
    drdt = rpunto
    drpunto_dt = np.dot(coef_vector,u)
    return drdt,drpunto_dt

Integramos las ecuaciones y graficamos

In [30]:
X0 = (x[0],x_punto[0])
print('Initial conditions: X = ',X0[0],' X_punto = ',X0[1])
t = np.linspace(0,64,100000)
t2 = np.linspace(0,64,len(x))
tray = odeint(f,X0,t).T

fig = plt.figure()

plt.plot(t,tray[0])
plt.plot(t2,x)


Initial conditions: X =  -2.689574574105655e-19  X_punto =  -0.04288473923798376


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
t = np.linspace(0,64,1000)
t2 = np.linspace(64,0,1000)

t = t2
X0 = (x[0],x_punto[0])
tray = odeint(f,X0,t).T

X1 = (-.1,.01)
tray1 = odeint(f,X1,t).T

X2 = (1,.01)
tray2 = odeint(f,X2,t).T

X3 = (-.5,.01)
tray3 = odeint(f,X3,t).T



plt.figure()
plt.plot(tray[0],tray[1])
plt.plot(tray1[0],tray1[1])
plt.plot(tray2[0],tray2[1])
plt.plot(tray3[0],tray3[1])

a = tray[:,-1]
a2 = tray3[:,-1]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
t = np.linspace(0,1000,10000)
ic = [a + 0.4*(np.random.random(2) - 1/2) for i in range(10)]
plt.close('all')
plt.figure(90)
for i in ic:
    tray = odeint(f,i,t).T
    plt.plot(tray[0],tray[1])
    
t = np.linspace(0,1000,10000)
ic = [a2 +.5*(np.random.random(2) - 1/2) for i in range(10)]
for i in ic:
    tray = odeint(f,i,t).T
    plt.plot(tray[0],tray[1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

EL foco es un foco repulsor

In [33]:
t = np.linspace(0,100,100000)
ic = [a + 0.4*(np.random.random(2) - 1/2) for i in range(1)]
plt.close('all')
plt.figure(90)
for i in ic:
    tray = odeint(f,i,t).T
    plt.plot(t,tray[0])

ic = [a2 +1*(np.random.random(2) - 1/2) for i in range(1)]
for i in ic:
    tray = odeint(f,i,t).T
    plt.plot(t,tray[0])
plt.plot(t,x)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Una neurona mas

In [34]:
Cluster_selected = get_cluster_indexs(24,fl)
Emb = Neurons.loc[Cluster_selected]
x = np.concatenate(Emb.Mean.values)
x = x[1:64]
x = (x - np.mean(x))/np.std(x)
x = x - x[0]


In [35]:

t = np.linspace(0,len(x),100000)
t2 = np.arange(len(x))
x_interp = UnivariateSpline(t2,x,s = 0)
x_punto_interp = x_interp.derivative()
x_puntopunto_interp = x_interp.derivative(2)

x_interp = x_interp(t)
x_punto_interp = x_punto_interp(t)
x_puntopunto_interp = x_puntopunto_interp(t)


x_punto = np.gradient(x)
x_puntopunto = np.gradient(x_punto)


x_punto_integrated = cumtrapz(x_puntopunto)
x_integrated = cumtrapz(x_punto_integrated)

x_punto_integrated_interp= cumtrapz(x_puntopunto_interp)
x_integrated_interp= cumtrapz(x_punto_integrated_interp)

fig = plt.figure()
axes = fig.subplots(2,2)
axes[0][0].scatter(t2,x,label = 'senial original')
axes[0][0].plot(x_integrated,color = 'r',label = 'senial integrada')
axes[0][1].scatter(t,x_interp,color = 'g', marker = '.',label = 'senial interpolada')
axes[0][1].plot(t[1:-1],x_integrated_interp,color = 'c',label = 'senial interpolada integrada')

axes[1][0].plot(x_punto,label = 'x_punto')
axes[1][0].plot(x_puntopunto,label = 'x_puntopunto')
axes[1][1].plot(t,x_punto_interp,label = 'x_punto interp')
axes[1][1].plot(t,x_puntopunto_interp,label = 'x_puntopunto interp')
axes[0][0].legend()
axes[0][1].legend()
axes[1][0].legend()
axes[1][1].legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [36]:
x = x_interp
x_punto = x_punto_interp
x_puntopunto = x_puntopunto_interp
degree = 10
idx = np.arange(1,degree,2)
Theta = np.zeros([x.shape[0],degree])
Theta[:,0] = np.ones(x.shape[0])
Theta[:,1] = x
Theta[:,2] = x_punto
Theta[:,3] = x*x_punto
Theta[:,4] = x_punto**2
Theta[:,5] = x**3
Theta[:,6] = x**2*x_punto
Theta[:,7] = x**2
Theta[:,8] = x_punto**3
Theta[:,9] = x**4

Lasso

In [37]:
from sklearn.linear_model import Lasso
from scipy.integrate import odeint
classifier = Lasso(alpha=0.0000001,max_iter=100000)
classifier.fit(Theta,x_puntopunto)
print(Theta.shape,len(x_puntopunto))

(100000, 10) 100000


/home/lorenzo/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 21.04400919268219, tolerance: 0.13212147197042645
  positive)


Vector de coeficientes ($\eta$)

In [38]:
coef_vector = classifier.coef_
print(coef_vector)

[ 0.         -0.05209535  0.34361056  0.28813098 -0.58926565 -0.10090937
  0.04591659 -0.16573967 -0.2793489  -0.01342172]


Ecuaciones diferenciales

In [39]:
def f(X,t):
    r,rpunto = X
    u = np.array([1,r,rpunto,r*rpunto,rpunto**2,r**3,rpunto*r**2,r**2,rpunto**3,r**4])
    
    drdt = rpunto
    drpunto_dt = np.dot(coef_vector,u)
    return drdt,drpunto_dt

Integramos las ecuaciones y graficamos

In [40]:
X0 = (-.001, 0.000003)
print('Initial conditions: X = ',X0[0],' X_punto = ',X0[1])
t = np.linspace(0,64,10000)
t2 = np.linspace(0,64,len(x))
tray = odeint(f,X0,t).T

fig = plt.figure()

plt.plot(t,tray[0])
plt.plot(t2,x)


Initial conditions: X =  -0.001  X_punto =  3e-06


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
t = np.linspace(0,1,1000)
t2 = np.linspace(150,0,1000)


X0 = (x[0],x_punto[0])
tray = odeint(f,X0,t2).T

X1 = (-.1,.01)
tray1 = odeint(f,X1,t2).T

X2 = (1,.01)
tray2 = odeint(f,X2,t).T

X3 = (-.5,.01)
tray3 = odeint(f,X3,t2).T



plt.figure()
plt.plot(tray[0],tray[1])
plt.plot(tray1[0],tray1[1])
plt.plot(tray2[0],tray2[1])
plt.plot(tray3[0],tray3[1])

a = tray[:,-1]
a2 = tray3[:,-1]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …